<a href="https://colab.research.google.com/github/lvaughan9/nft-scripts/blob/main/Basic_NFT_generation_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import random
import json
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

nft_path = drive_path / 'NFTs'
ASSETFOLDER = nft_path / 'assets'
OUTPUTFOLDER =  nft_path / 'NFT final images'
METADATAOUTPUTFOLDER = nft_path / 'metadata_to_upload'
WRAPPEROUTPUTFOLDER =  nft_path / 'metadata_for_chain'
%mkdir "{OUTPUTFOLDER}"

trait_sets = []     # Generated attribute sets (one per NFT)

dev_wallet_1 = 'WALLETADDRESS'
dev_share_1 = 100   # Share, in percent
dev_wallet_2 = 'WALLETADDRESS'
dev_share_2 = 0

background_dict = { 0 : 'Background 1', 1 : 'Background 2', 2 : 'Background 3', 3 : 'Background 4'}
attribute_one_dict = { 0 : 'None', 1 : 'Trait one', 2 : 'Trait two', 3 : 'Trait three'}
attribute_two_dict = { 0 : 'None', 1 : 'Trait one', 2 : 'Trait two', 3 : 'Trait three'}
attribute_three_dict = { 0 : 'None', 1 : 'Trait one', 2 : 'Trait two', 3 : 'Trait three'}

background_weights = (20,20,30,30)
attribute_one_weights = (20,20,30,30)
attribute_two_weights = (20,20,30,30)
attribute_three_weights = (20,20,30,30)

background_count = []
attribute_one_count = []
attribute_two_count = []
attribute_three_count = []

for i in range(len(background_dict)):
  background_count.append(0)
for i in range(len(attribute_one_dict)):
  attribute_one_count.append(0)
for i in range(len(attribute_two_dict)):
  attribute_two_count.append(0)
for i in range(len(attribute_three_dict)):
  attribute_three_count.append(0)

num_nfts = 10

def random_selector(type):
    if type == "background":
        num = random.choices(background_dict,background_weights)[0]
        return num
    if type == "attribute_one":
        num = random.choices(attribute_one_dict,attribute_one_weights)[0]
        return num
    if type == "attribute_two":
        num = random.choices(attribute_two_dict,attribute_two_weights)[0]
        return num
    if type == "attribute_three":
        num = random.choices(attribute_three_dict,attribute_three_weights)[0]
        return num

empty_metadata = {
}

def generate_meta_data_template(token_id):
    metadata = empty_metadata
    metadata["name"] = 'NFT # ' + str(token_id)
    metadata["seller_fee_basis_points"] = '500' # In 100ths of a percent, for some reason
    metadata["description"] = 'Collection Description'
    metadata["image"] = ''
    metadata["symbol"] = 'SYMBOL'
    metadata["animation_url"] = ''
    metadata["collection"] = {"name": "Collection","family": "Collection"}
    metadata["external_url"] = 'http://nft.website'
    metadata["attributes"] = []
    return metadata

def generate_random_traits(trait_sets[]):
  background = random_selector("background")
  attribute_one = random_selector("attribute_one")
  attribute_two = random_selector("attribute_one")
  attribute_three = random_selector("attribute_one")

  trait_set = (attribute_one, attribute_two, attribute_three)

  #Deal with collisions here.

  if trait_set in trait_sets:
    return generate_random_traits(trait_sets)   # Randomize again if this combo exists
    else:
      trait_sets.append(trait_set)              # If it doesn't exist already, append it to the list
      return trait_set

def create_image(trait_set, token_id):
    base_img = Image.open(ASSETFOLDER / "background" / (str(background) + ".png")) # We start with the background as our base image
    attribute_one_img = Image.open(ASSETFOLDER / "attribute_one" / (str(attribute_one) + ".png"))
    attribute_two_img = Image.open(ASSETFOLDER / "attribute_two" / (str(attribute_two) + ".PNG"))
    attribute_three_img = Image.open(ASSETFOLDER / "attribute_three" / (str(attribute_three) + ".PNG"))

    # Paste/Merge Required PNGs, as layers on base
    base_img.paste(attribute_one_img, (0, 0), attribute_one_img)
    base_img.paste(attribute_two_img, (0, 0), attribute_two_img)
    base_img.paste(attribute_three_img, (0, 0), attribute_three_img)

    #handle any special cases here

    #if attribute_one in (7,8) and attribute_two in (1,5,8,9): 
    #  base_img.paste(torso_img, (0, 0), torso_img)  #paste mask again

# Resize image to OpenSea market recommended size - "Resample Nearest" to retain resolution
    resized_img = base_img.resize((350, 350), resample=Image.NEAREST)
    resized_img.save(OUTPUTFOLDER / (str(token_id) + '.png'), "PNG")
  return

for token_id in range(num_nfts):
    (background,body,face,hat,accessory,special) = generate_random_traits(trait_sets[])

    # Debug line
    #print(str(background) + "," + str(attribute_one) + "," + str(attribute_two) + "," + str(attribute_three)

    background_count[background] += 1
    attribute_one_count[attribute_one] += 1
    attribute_two_count[attribute_two] += 1
    attribute_three_count[attribute_three] += 1

    metadata = generate_meta_data_template(token_id)
    metadata['attributes'].append(
                    {
                        'trait_type': 'Custom attribute',
                        'value': '0'
                    })
    metadata['attributes'].append(
                    {
                        'trait_type': 'Background',
                        'value': background_dict[background]
                    })
    metadata['attributes'].append(
                    {
                        'trait_type': 'Attribute One',
                        'value': attribute_one_dict[attribute_one]
                    })
    metadata['attributes'].append(
                    {
                        'trait_type': 'Attribute Two',
                        'value': attribute_two_dict[attribute_two]
                    })
    metadata['attributes'].append(
                    {
                        'trait_type': 'Attribute Three',
                        'value': attribute_three_dict[attribute_three]
                    })
    png_name = (str(token_id) + '.png')
    metadata["properties"] = {'files':[{'uri':png_name,'type':'image/png'}],'category':'image','creators':[{'address':dev_wallet_1, 'share':dev_share_1},{'address':dev_wallet_2,'share':dev_share_2}]}
    metadata["image"] = png_name
    print(metadata)
    with open(OUTPUTFOLDER / (str(token_id) + ".json"), 'w') as outfile:
      json.dump(metadata, outfile)
    #data_list.append(metadata)

for i in range(len(attribute_one_dict)):
  print(accessory_dict[i] + ": " +  str(accessory_count[i]))


